In [0]:
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from zipfile import ZipFile
f = ZipFile('drive/My Drive/Dataset/classorig.zip', 'r')
f.extractall()

In [4]:
import os
os.listdir('actualdata/CONTENT')

['O', 'R']

In [5]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1
    )

train_generator = train_datagen.flow_from_directory(
    'actualdata/CONTENT',  
    target_size=(150, 150), 
    batch_size=32,
    class_mode='categorical',
    subset='training'
    )
test_generator = train_datagen.flow_from_directory(
    'actualdata/CONTENT',
    target_size=(150,150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
    )

Found 22570 images belonging to 2 classes.
Found 2507 images belonging to 2 classes.


In [0]:
base_model = tf.keras.applications.InceptionResNetV2(input_shape=(150,150,3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False                                   

In [0]:
model = tf.keras.models.Sequential([base_model,
                                    tf.keras.layers.Conv2D(32, (2,2), activation = 'relu'),
                                    tf.keras.layers.MaxPooling2D(pool_size=(1, 1)),
                                    tf.keras.layers.GlobalMaxPooling2D(),
                                    tf.keras.layers.Dense(2,activation = 'softmax')])

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [14]:
my_callbacks = [
    # tf.keras.callbacks.EarlyStopping(monitor='accuracy',patience=5,mode = 'max'),
    tf.keras.callbacks.ModelCheckpoint(monitor = 'accuracy',filepath='inceptiong-{accuracy:.4f}.h5',save_best_only=True,mode='max',),
]
model.fit(train_generator,
        batch_size=2000,
        steps_per_epoch=125,
        epochs=40,
        callbacks=my_callbacks
        )

Epoch 1/40
125/125 [==============================] - 24s 194ms/step - loss: 0.9288 - accuracy: 0.7856
Epoch 2/40
125/125 [==============================] - 25s 197ms/step - loss: 0.4117 - accuracy: 0.8813
Epoch 3/40
125/125 [==============================] - 25s 197ms/step - loss: 0.3917 - accuracy: 0.8850
Epoch 4/40
125/125 [==============================] - 25s 196ms/step - loss: 0.3661 - accuracy: 0.8938
Epoch 5/40
125/125 [==============================] - 24s 196ms/step - loss: 0.3554 - accuracy: 0.8945
Epoch 6/40
125/125 [==============================] - 25s 197ms/step - loss: 0.2814 - accuracy: 0.9005
Epoch 7/40
125/125 [==============================] - 24s 196ms/step - loss: 0.2660 - accuracy: 0.9050
Epoch 8/40
125/125 [==============================] - 24s 196ms/step - loss: 0.2587 - accuracy: 0.9103
Epoch 9/40
125/125 [==============================] - 25s 196ms/step - loss: 0.2464 - accuracy: 0.9120
Epoch 10/40
125/125 [==============================] - 25s 199ms/step - l

In [22]:
nmodel = tf.keras.models.load_model('inceptiong-0.9392.h5')
nmodel.evaluate(test_generator)

79/79 [==============================] - 15s 186ms/step - loss: 0.2886 - accuracy: 0.9055


[0.2886269986629486, 0.9054647088050842]